In [ ]:
# Header Files
import pandas as pd
import csv
import numpy as np
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from csv import DictReader
from efficient_apriori import apriori as eff_app
import ast
from mlxtend.preprocessing import TransactionEncoder
from nltk.stem import WordNetLemmatizer
from mlxtend.frequent_patterns import apriori

### Step 1.  Data Preprocessing ###

In [ ]:
# Using Kaggle DataSet 
trainRecipe = []
included_cols = [1, 10]
count = 100000
lemmatizer = WordNetLemmatizer() 
with open('C:/Users/Mayank/Documents/Project/RAW_recipes.csv', encoding="utf8") as f:
    read =csv.reader(f)
    header = next(read)
    for r in read:            
        recipe = list(r[i] for i in included_cols)
        #print(type(recipe[1])) '<class>, str'        
        ingr = ast.literal_eval(recipe[1])
        ingrStem = [' '.join(stem_tokens(tokenize(w))) for w in ingr]
        trainRecipe.append(ingrStem)
        count -= 1
        if(count == 0):
            break

### Step 2. Apriori ###

In [ ]:
itemsets, rules = eff_app(trainRecipe, min_support=0.2,  min_confidence=1)
print((itemsets))
print("Rules =>", rules)

In [ ]:
print(len(trainRecipe+training_set))
comb = trainRecipe+training_set
print(comb[99998:100005])

In [ ]:
from itertools import chain
c = Counter(chain.from_iterable(comb)).most_common()[:-30-1:-1]
print("c = ",c)

te = TransactionEncoder()

te_ary = te.fit(comb).transform(comb)

df = pd.DataFrame(te_ary, columns=te.columns_)
#print((df.columns))
cls2 = apriori(df, min_support=0.1, use_colnames = True)
#print((cls2))
#print(df)
#print(df[cls2['itemsets']])

In [ ]:
def stem_tokens(tokens):
    stemmed_items = []
    for items in tokens:
        stemmed_items.append(lemmatizer.lemmatize(items))
    return stemmed_items

def tokenize(text):
    
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems

import nltk
str1 = "ziplock bags"  
tok = tokenize(str1)
print(' '.join(stem_tokens(tok)))

In [ ]:

import nltk
nltk.download('wordnet')

In [ ]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat, idx

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

In [164]:
#Using dataset from https://eightportions.com/datasets/Recipes/
#Contains recipe name and list of ingredients used
data1 = pd.read_json('data/recipes_raw_nosource_ar.json', orient = 'index')
recipeIg1 = data1['ingredients']
len1 = len(recipeIg1)

data2 = pd.read_json('data/recipes_raw_nosource_epi.json', orient = 'index')
recipeIg2 = data2[['title','ingredients']]
len2 = len(recipeIg2)

data3 = pd.read_json('data/recipes_raw_nosource_fn.json', orient = 'index')
recipeIg3 = data2[['title','ingredients']]
len3 = len(recipeIg3)

print(len1+len2+len3)
print(recipeIg3)

90448
                                                                             title  \
05zEpbSqcs9E0rcnCJWyZ9OgdH0MLby                                  Christmas Eggnog    
mF5SZmoqxF4WtIlhLRvzuKk.z6s7P2S                  Veal, Carrot and Chestnut Ragoût    
oQV5D7cVbCFwmrDs3pBUv2y.AG0WV26  Caramelized Bread Pudding with Chocolate and C...   
Z9seBJWaB5NkSp4DQHDnCAUBTwov/1u      Sherried Stilton and Green Peppercorn Spread    
bB3GxoAplVZeoX3fzWNWyeECtQFxw6G                        Almond-Chocolate Macaroons    
FHQAJvovVtPyKWlzgFEHgSUJsCM2Tjq                     White Sauce or Bechamel Sauce    
Jt7RKsNPY7/1QrOjuPEomP/s6mD2AvO  Grilled Whole Mackerel with Lemon, Oregano, an...   
1rBWKEw7vZjXA97NmHGS3VHq1iVX9c.                                   Apricot Chutney    
agTMHTysNlAOyMM3zQlXVu4qbJrLmqa                                   Garlic Croutons    
Y..dFCvcwNW4gdogZ3DqLUETR2x.uA2                           Christmas Croquembouche    
wAybSELYww/.RDBrZyskjgx0XueW1Mq                 

In [ ]:
#Data URL: https://www.kaggle.com/kaggle/recipe-ingredients-dataset(Data set with cuisines)

# Create URL to JSON file (alternatively this can be a filepath)
url = 'C:/Users/Mayank/Documents/Project/train.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

ingredients_1 = df['ingredients']
# View the first ten rows
train_list = ingredients_1.values.tolist()
print(train_list[0:10])
print(ingredients_1.head(10))
len(ingredients_1)
training_set = []
for ingr in ingredients_1:
    ingrStem = [' '.join(stem_tokens(tokenize(w))) for w in ingr]
    training_set.append(ingrStem)

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(train_list).transform(train_list)
df = pd.DataFrame(te_ary, columns=te.columns_)
#print((df.columns))
cls2 = apriori(df, min_support=0.1, use_colnames = True)
print((cls2))
#print(df)
#print(df[cls2['itemsets']])

In [ ]:
#Data URL: https://www.kaggle.com/hugodarwood/epirecipes

# Create URL to JSON file (alternatively this can be a filepath)
url = 'full_format_recipes.json'

# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

ingredients_2 = df[['title','ingredients']]
# View the first ten rows
print(ingredients_2.head(10))
print(len(ingredients_2))

In [ ]:
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

with np.load('C:/Users/Mayank/Documents/Project/simplified-recipes-1M.npz') as data:
    recipes = data['recipes']
    ingredients = data['ingredients']

In [ ]:
ingrlist = []
counter = 0
for recipe in recipes:
    if(recipe.size) == 0:
        continue
    ingrlist.append(ingredients[recipe])
    counter += 1
    #break

print(counter)
#ingredients[recipes[0]]
#print(ingrlist)

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(ingrlist).transform(ingrlist)
df = pd.DataFrame(te_ary, columns=te.columns_)
#print((df.columns))
cls2 = apriori(df, min_support=0.2, use_colnames = True)
print((cls2))

In [ ]:
itemsets, rules = eff_app(ingrlist, min_support=0.08,  min_confidence=0.3)
print((itemsets))
print("Rules =>", rules)